2018 / 2 / 01

Wayne Nixalo

# RetinaNet Finetuning

## 1. Imports & Initialization

In [1]:
import keras
import keras.preprocessing.image

import keras_retinanet.losses
import keras_retinanet.layers
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.models.resnet import ResNet50RetinaNet
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.models.resnet import custom_objects # <-- what do I need this for?

import tensorflow as tf

import numpy as np
import os
import cv2

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

Taken from [Boring Detector train.py script](https://github.com/lexfridman/boring-detector/blob/master/train.py#L43)

Based on [initialization API from RetinaNet](https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/models/resnet.py#L100)

In [3]:
def create_models(num_classes):
    # create "base" model (no NMS)
    image = keras.layers.Input((None, None, 3))
    
    model = ResNet50RetinaNet(image, num_classes=num_classes, weights='imagenet', nms=False)
    training_model = model
    
    # append NMS for prediction only
    classification   = model.outputs[1]
    detections       = model.outputs[2]
    boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
    detections       = keras_retinanet.layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
    prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    
    # compile model
    training_model.compile(
        loss={
            'regression'    : keras_retinanet.losses.smooth_l1(),
            'classification':keras_retinanet.losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model, training_model, prediction_model

In [4]:
# keras.backend.tensorflow_backend.set_session(get_session())

# model = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
#                                     custom_objects=custom_objects)

In [5]:
LENDATSET = 7637 # length of dataset (7637 images)
batch_size = 32

num_classes = 1
steps_per_epoch = LENDATSET // batch_size # may need edits for batch sizes that leave remainders

In [6]:
image_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
datagen = CSVGenerator(csv_data_file='data/interstage_retinanet_data.csv',
                       csv_class_file='data/interstage_retinanet_classes.csv',
                       image_data_generator=image_gen,
                       batch_size=batch_size)

In [7]:
model, trainer_model, predictor_model = create_models(num_classes)
# dont yet exactly know why it's done this way: 3 returned 'model' versions

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_2" during training.


## 2. Training

In [18]:
trainer_model.optimizer.lr.value()

<tf.Tensor 'Adam/lr/read:0' shape=() dtype=float32>

Oh ffs Keras just show me the fucking learning rate.

In [11]:
# recompile training-model to change learning rate (as needed)
trainer_model.compile(
    loss={
        'regression'    : keras_retinanet.losses.smooth_l1(),
        'classification':keras_retinanet.losses.focal()
    },
    optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
)

<tf.Variable 'Adam/lr:0' shape=() dtype=float32_ref>

In [19]:
trainer_model.fit_generator(generator = datagen,
                            steps_per_epoch = steps_per_epoch, # eh?
                            epochs    = 1,
                            verbose   = 1,)

Epoch 1/1


/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 5690 (shape (400, 400, 3)) contains the following invalid boxes: [array([ 59., 129., 316., 401.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]
/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 5691 (shape (400, 400, 3)) contains the following invalid boxes: [array([ 63., 130., 314., 402.,   0.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]
/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 5692 (shape (400, 400, 3)) contains the following invalid boxes: [array([101.,  81., 260., 405.,   0.])].
  [annotations[invalid_inde

ResourceExhaustedError: OOM when allocating tensor with shape[32,150,150,256]
	 [[Node: res2a_branch1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](pool1/MaxPool, res2a_branch1/kernel/read)]]
	 [[Node: loss/classification_loss/Mean_2/_2065 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_9662_loss/classification_loss/Mean_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'res2a_branch1/convolution', defined at:
  File "/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/

## 3. Testing & Visualization

In [ ]:
test_image_path = 'data/interstage_train/000000-000412/000001.jpg' # or script for random impath
test_image = cv2.imread(test_image_path)

prediction = predictor_model.predict_on_batch(np.expand_dims(test_image, axis=0))

In [ ]:
# preprocess image for neural network
image = preprocess_image(test_image)
image, scale = resize_image(image)

# detect on image
_,_,detections = model.predict_on_batch(np.expand_dims(image, axis=0))

# compute predicted labels and scores
detected_label = np.argmax(detections[0,:,4:], axis=1)
scores         = detections[0, np.arange(detections.shape[1]), 4 + detected_label]

# correct for image scale
detections[0,:,:4] /= scale

In [ ]:
bbx = detections[0, 0, :4]
bbx